<a href="https://colab.research.google.com/github/24p11/recode-icd/blob/main/postprocess_mistral_report_generation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
import json
import ast
import re

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
experience_name = "sample_10000_20250924"
file_name = "crh_sample_10000_20250924.csv"

In [ ]:
path_ref = "/content/drive/MyDrive/Results/clinical_reports/" + experience_name + "/"

In [ ]:
df_gen_reports = pd.read_csv(path_ref + file_name, index_col=0).reset_index(drop=True)

In [ ]:
df_gen_reports.columns

Index(['age', 'cage', 'cage2', 'sexe', 'date_entry', 'date_discharge',
       'date_of_birth', 'first_name', 'last_name', 'icd_primary_code',
       'icd_primary_description', 'icd_parent_code', 'case_management_type',
       'case_management_type_description', 'coding_rule',
       'case_management_type_text', 'drg_parent_code',
       'drg_parent_description', 'icd_secondaray_code',
       'text_secondary_icd_official', 'procedure', 'text_procedure',
       'admission_type', 'admission_mode', 'discharge_disposition', 'dms',
       'los_mean', 'los_sd', 'cancer_stage', 'score_TNM', 'histological_type',
       'treatment_recommandation', 'chemotherapy_regimen', 'biomarkers',
       'first_name_med', 'last_name_med', 'department', 'hospital',
       'template_name', 'user_prompt', 'system_prompt', 'prefix', 'prefix_len',
       'bacth', 'num_in_', 'crh'],
      dtype='object')

In [ ]:
print(df_gen_reports.loc[0,"crh"])


{
  "CR": "
**HÔPITAL DE HAUTEPIERRE - HÔPITAUX UNIVERSITAIRES DE STRASBOURG**
**SERVICE DE RADIOTHÉRAPIE**

---
**IDENTITÉ DU PATIENT**
Nom : RUPPA
Prénom : Emelia
Date de naissance : 23/02/1963
---

### Motif d’hospitalisation
Madame Emelia Ruppa, 61 ans, est hospitalisée du **13/09/2024 au 18/09/2024** dans le service de radiothérapie pour la réalisation d’une séance de radiothérapie cérébrale. Cette prise en charge s’inscrit dans le cadre du traitement de ses métastases cérébrales et cutanées, secondaires à un cancer du poumon diagnostiqué en 2022. L’hospitalisation est programmée pour la préparation et l’administration d’une irradiation externe avec repérage scannographique et dosimétrie 3D, dans un contexte de progression tumorale malgré les lignes de traitement antérieures.

---

### Antécédents
**Médicaux** :
- **Cancer bronchique non à petites cellules** (adénocarcinome) diagnostiqué en **juin 2022**, stade IV d’emblée (cT4N1M1) avec métastases cérébrales et osseuses. Mutatio

In [ ]:
import re
import json

def extract_json(text):
  match = re.search(r"```json(.*?)```", text, re.DOTALL)
  if match:
    return match.group(1).strip()
  else:
    return None

def clean_bold(text):
  return re.sub(r"\*\*(.*?)\*\*", r"\1", text)

def clean_header(text):
  clean_text = re.sub(r'^##+ .*$', '', text, flags=re.MULTILINE)
  clean_text = re.sub(r'^--+$', '', clean_text, flags=re.MULTILINE)
  return clean_text

def delete_comments(json_text):
  """
  There are a few annotations (values of the "formulations" key of the dictionary) that contain comments of the format: // or /* */, which make `json.loads` fail.
  This function aims to fix them.
  """
  json_text = re.sub(r'//.*', '', json_text)
  json_text = re.sub(r'/\*.*?(?:\*/|\n)', '', json_text, flags=re.DOTALL)
  return json_text

def fix_multiline_strings(json_text):
  """
  There are a few CRs (values of the "CR" key of the dictionary) that contain new lines in bad formats, which make `json.loads` fail.
  This function aims to fix them.
  """
  def replacer(match):
    content = match.group(1)
    content = (content
               .replace('"', '\\"')
               .replace('\\\\"', '\\"')
               .replace("\n", "\\n"))
    formulation_wording = match.group(2)
    return f'"CR": "{content}"{formulation_wording}'

  fixed = re.sub(r'"CR":\s*"(.*?)"(,\s*"formulations")', replacer, json_text, flags=re.DOTALL)
  return fixed
# import ast

def extract_generations_annotations(response):
  response = extract_json(response)
  response = fix_multiline_strings(response)
  response = delete_comments(response)
  try:
    response_dict = json.loads(response)
  except json.JSONDecodeError:
    return (None, None, None)

  if isinstance(response_dict, dict):
    if ("CR" in response_dict) and ("formulations" in response_dict):
      if isinstance(response_dict["formulations"], dict):
        if ("diagnostics" in response_dict["formulations"]) and ("informations" in response_dict["formulations"]):
          return (clean_bold(clean_header(response_dict["CR"])).strip(), response_dict["formulations"]["diagnostics"], response_dict["formulations"]["informations"])
        else:
          return (None, None, None)
      else:
        return (None, None, None)
    else:
      return (None, None, None)
  else:
    return (None, None, None)

In [ ]:
import re
import json

def extract_json(text):
  match = re.search(r"```json(.*?)```", text, re.DOTALL)
  if match:
    return match.group(1).strip()
  else:
    return None

def clean_bold(text):
  return re.sub(r"\*\*(.*?)\*\*", r"\1", text)

def clean_header(text):
  clean_text = re.sub(r'^##+ .*$', '', text, flags=re.MULTILINE)
  clean_text = re.sub(r'^--+$', '', clean_text, flags=re.MULTILINE)
  return clean_text

def delete_comments(json_text):
  """
  There are a few annotations (values of the "formulations" key of the dictionary) that contain comments of the format: // or /* */, which make `json.loads` fail.
  This function aims to fix them.
  """
  json_text = re.sub(r'//.*', '', json_text)
  json_text = re.sub(r'/\*.*?(:?\*/|\n)', '', json_text, flags=re.DOTALL)
  return json_text

def fix_multiline_strings(json_text):
  """
  There are a few CRs (values of the "CR" key of the dictionary) that contain new lines in bad formats, which make `json.loads` fail.
  This function aims to fix them.
  """
  def replacer(match):
    content = match.group(1)
    content = (content
               .replace('"', '\"')
               .replace('\\"', '\"')
               .replace("\n", "\\n")
               .replace("\t", "\\t")) # Added tab escaping
    formulation_wording = match.group(2)
    return f'"CR": "{content}"{formulation_wording}'

  fixed = re.sub(r'"CR":\s*"(.*?)"(,\s*"formulations")', replacer, json_text, flags=re.DOTALL)
  return fixed

def extract_generations_annotations(json_string_input):
  # The input 'json_string_input' is assumed to be the raw JSON string from 'crh' column.
  # The extract_json function is not needed here as the data is already raw JSON.

  if not isinstance(json_string_input, str):
    return (None, None, None)

  # Apply fixes for multiline strings and comments
  fixed_multiline_string = fix_multiline_strings(json_string_input)
  no_comments_string = delete_comments(fixed_multiline_string)

  try:
    response_dict = json.loads(no_comments_string)
  except json.JSONDecodeError:
    return (None, None, None)

  if isinstance(response_dict, dict):
    if ("CR" in response_dict) and ("formulations" in response_dict):
      if isinstance(response_dict["formulations"], dict):
        if ("diagnostics" in response_dict["formulations"]) and ("informations" in response_dict["formulations"]):
          # Ensure CR content is handled, then clean bold/header
          cr_content = response_dict["CR"]
          processed_cr = clean_bold(clean_header(cr_content)).strip()
          return (processed_cr, response_dict["formulations"]["diagnostics"], response_dict["formulations"]["informations"])
        else:
          return (None, None, None)
      else:
        return (None, None, None)
    else:
      return (None, None, None)
  else:
    return (None, None, None)

In [ ]:
df_gen_reports["clinical_note"] = ""
df_gen_reports["annot_diagnostics"] = ""
df_gen_reports["annot_criteria"] = ""

for i in range(len(df_gen_reports)):
  response = df_gen_reports.iloc[i]["crh"]
  clinical_note, annot_diag, annot_criteria = extract_generations_annotations(response)
  df_gen_reports.at[i, "clinical_note"] = clinical_note
  df_gen_reports.at[i, "annot_diagnostics"] = annot_diag
  df_gen_reports.at[i, "annot_criteria"] = annot_criteria

In [ ]:
print(df_gen_reports.loc[0, "clinical_note"])

HÔPITAL DE HAUTEPIERRE - HÔPITAUX UNIVERSITAIRES DE STRASBOURG
SERVICE DE RADIOTHÉRAPIE


IDENTITÉ DU PATIENT
Nom : RUPPA
Prénom : Emelia
Date de naissance : 23/02/1963



Madame Emelia Ruppa, 61 ans, est hospitalisée du 13/09/2024 au 18/09/2024 dans le service de radiothérapie pour la réalisation d’une séance de radiothérapie cérébrale. Cette prise en charge s’inscrit dans le cadre du traitement de ses métastases cérébrales et cutanées, secondaires à un cancer du poumon diagnostiqué en 2022. L’hospitalisation est programmée pour la préparation et l’administration d’une irradiation externe avec repérage scannographique et dosimétrie 3D, dans un contexte de progression tumorale malgré les lignes de traitement antérieures.




Médicaux :
- Cancer bronchique non à petites cellules (adénocarcinome) diagnostiqué en juin 2022, stade IV d’emblée (cT4N1M1) avec métastases cérébrales et osseuses. Mutation EGFR exon 19 del (détection par NGS sur biopsie pulmonaire).
- Hypertension artérielle tra

In [ ]:
print(df_gen_reports.loc[0, "annot_diagnostics"])

{'Tumeur maligne secondaire du cerveau et des méninges cérébrales (C793)': ['métastases cérébrales', 'lésions métastatiques cérébrales', 'métastases cérébrales progressives', '3 lésions métastatiques (frontale gauche 25 mm, pariétale droite 18 mm, occipitale gauche 12 mm)'], 'Hémiplégie, sans précision (G819)': ['hémiplégie droite', 'hémiplégie droite avec spasticité', 'score MRC 2/5 bras, 3/5 jambe', 'déambulateur sur 10 mètres max'], 'Hypertension essentielle (primitive) (I10)': ['hypertension artérielle', 'poussée d’HTA', 'PA 145/88 mmHg', 'épisode d’HTA à 160/95 mmHg'], 'État grabataire (R2630)': ['alitée une partie de la journée', 'état grabataire partiel', 'fatigue marquée (sieste de 3h l’après-midi)'], 'Tumeur maligne secondaire de la peau (C792)': ['métastases cutanées', 'nodules violacés du cuir chevelu', 'plaque érythémateuse de la région claviculaire gauche', 'lésions cutanées suintantes'], "Tumeur maligne secondaire d'autres sièges précisés (C798)": ['métastases hépatiques 

In [ ]:
print(df_gen_reports.loc[0, "annot_criteria"])

{'Date entrée': ['13/09/2024'], 'Date de sortie': ['18/09/2024'], "Service d'hospitalisation": ['Service de Radiothérapie – Hôpital de Hautepierre'], 'Nom/Prénom du médecin': ['Dr Amaya Frayssinet'], 'Nom/Prénom du patient': ['Madame Emelia Ruppa'], 'Âge': ['61 ans'], 'Sexe': ['Féminin'], 'État général': ['Score ECOG 3', 'altération de l’état général', 'patient fatigué, coopérant, mais avec ralentissement idéomoteur'], 'Poids': ['52 kg'], 'Statut gestationnel': ['ménopause à 52 ans'], 'Gestité': ['G3P3'], 'Topographie de la tumeur primaire': ['cancer bronchique non à petites cellules', 'adénocarcinome pulmonaire'], 'Histologie tumorale': ['adénocarcinome', 'mutation EGFR exon 19 del'], 'Stade tumoral': ['stade IV d’emblée (cT4N1M1)', 'progression tumorale malgré les lignes de traitement antérieures'], 'Biomarqueurs tumoraux': ['mutation EGFR exon 19 del (détection par NGS sur biopsie pulmonaire)'], 'Sites métastatiques': ['métastases cérébrales, osseuses et surrénaliennes (diagnostic i

In [ ]:
df_icd_official = pd.read_excel("/content/drive/MyDrive/Referentials/cim_2024.xlsx")
#transform dict and list after import :


df_icd_official = df_icd_official.explode("code")

In [ ]:
def get_icd_official(code):
  return df_icd_official[df_icd_official["code"] == code]["libelle"].iloc[0]

In [ ]:
def delete_comments(json_text):
  """
  There are a few annotations (values of the "formulations" key of the dictionary) that contain comments of the format: // or /* */, which make `json.loads` fail.
  This function aims to fix them.
  """
  json_text = re.sub(r'//.*', '', json_text)
  json_text = re.sub(r'/\*.*?(?:\*/|\n)', '', json_text, flags=re.DOTALL)
  return json_text

In [ ]:
import re
"""
There are a few annotation of the ICD codes whose labels contain only the codes and not don't contain the definitions of the codes.
This piece of code aims to fix them.
"""

for i in range(0,len(df_gen_reports)):
  annot_diag = df_gen_reports.iloc[i]["annot_diagnostics"]
  if annot_diag is not None:
    new_dict = {}
    for key in annot_diag.keys():
      if "[...]" in key:
        # Try to find a code in parentheses first, e.g., 'Description (CODE)'
        match_parentheses = re.search(r'\(([A-Z]\d{2,5}\+?\d*)\)', key)
        code = None
        if match_parentheses:
          code = match_parentheses.group(1)
        else:
          # If not found in parentheses, try to find code at the beginning, e.g., 'CODE - Description'
          match_start = re.match(r"^([A-Z]\d{2,5}\+?\d*)\s*-", key)
          if match_start:
            code = match_start.group(1)

        if code: # If a code was successfully extracted
          oficial_name = get_icd_official(code)
          new_dict[code + " - " + oficial_name] = annot_diag[key]
        else: # If no code could be extracted, keep the original key
          new_dict[key] = annot_diag[key]
      else:
        new_dict[key] = annot_diag[key]
    df_gen_reports.at[i, "annot_diagnostics"] = new_dict

In [ ]:
print(df_gen_reports.loc[0, "annot_diagnostics"])

{'Tumeur maligne secondaire du cerveau et des méninges cérébrales (C793)': ['métastases cérébrales', 'lésions métastatiques cérébrales', 'métastases cérébrales progressives', '3 lésions métastatiques (frontale gauche 25 mm, pariétale droite 18 mm, occipitale gauche 12 mm)'], 'Hémiplégie, sans précision (G819)': ['hémiplégie droite', 'hémiplégie droite avec spasticité', 'score MRC 2/5 bras, 3/5 jambe', 'déambulateur sur 10 mètres max'], 'Hypertension essentielle (primitive) (I10)': ['hypertension artérielle', 'poussée d’HTA', 'PA 145/88 mmHg', 'épisode d’HTA à 160/95 mmHg'], 'État grabataire (R2630)': ['alitée une partie de la journée', 'état grabataire partiel', 'fatigue marquée (sieste de 3h l’après-midi)'], 'Tumeur maligne secondaire de la peau (C792)': ['métastases cutanées', 'nodules violacés du cuir chevelu', 'plaque érythémateuse de la région claviculaire gauche', 'lésions cutanées suintantes'], "Tumeur maligne secondaire d'autres sièges précisés (C798)": ['métastases hépatiques 

### Calculate entities span

In [ ]:
def tokenize_text(text):
  return re.findall(r'\w+(?:[-_]\w+)*|\S', text)

In [ ]:
def count_tokens(labels, text, max_len=384):
  nb_tokens_labels = 0
  for label in labels:
    nb_tokens_labels += len(tokenize_text(label))

  nb_tokens_text = len(tokenize_text(text))

  return nb_tokens_labels + nb_tokens_text <= max_len

In [ ]:
def filter_nested_entities_with_same_labels(entities_spans):
  entities_spans = sorted(entities_spans, key=lambda x: (x[2], x[0], -(x[1]-x[0])))

  filtered = []
  for start, end, label in entities_spans:
    nested = False
    for fs, fe, fl in filtered:
        if fl == label and fs <= start and end <= fe:
            nested = True
            break
    if not nested:
        filtered.append((start, end, label))

  return filtered

In [ ]:
def filter_annotations(annotations, text):
  """
  There are a few annotations whose text spans are "aucun", "aucune", "aucuns" or "aucunes", which are not the correct text spans for the corresponding labels (They are just interpretations from Mistral, meaning that there are no entities for those labels in the text).
  This function aims to remove those kinds of annotations.
  """
  tokenized_text = tokenize_text(text)
  entity_spans = []

  for label, list_spans in annotations.items():
    matches = []
    for span in list_spans:
      if ("aucun" not in span.lower()) and ("aucune" not in span.lower()) and ("aucuns" not in span.lower()) and ("aucunes" not in span.lower()):
        tokenized_span = tokenize_text(span)
        for i in range(len(tokenized_text) - len(tokenized_span) + 1):
          if " ".join(tokenized_text[i:i + len(tokenized_span)]).lower() == " ".join(tokenized_span).lower():
            matches.append((i, i + len(tokenized_span) - 1, label))

    if matches:
      entity_spans.extend(matches)
  return filter_nested_entities_with_same_labels(entity_spans)

In [ ]:
def process_text(annotations, text):
  tokenized_text = tokenize_text(text)
  entity_spans = filter_annotations(annotations, text)
  list_labels = list(set([span[2] for span in entity_spans]))

  return tokenized_text, entity_spans, list_labels

In [ ]:
# Test export entity span
text = df_gen_reports.clinical_note.iloc[i]
annotations = {**df_gen_reports.iloc[i]["annot_diagnostics"], **df_gen_reports.iloc[i]["annot_criteria"]}
tokenized_subtext, entity_spans, list_labels = process_text(annotations, text)

In [ ]:
entity_spans

[(1167, 1177, 'Bilan hépatique'),
 (1179, 1184, 'Bilan hépatique'),
 (1186, 1190, 'Bilan hépatique'),
 (163, 175, 'Biomarqueurs tumoraux'),
 (1150, 1161, 'Créatinine'),
 (45, 49, 'Date de sortie'),
 (1687, 1691, 'Date de sortie'),
 (39, 43, 'Date entrée'),
 (1266, 1270, 'Date entrée'),
 (379, 379, 'Gestité'),
 (142, 142, 'Histologie tumorale'),
 (163, 167, 'Histologie tumorale'),
 (459, 459, 'Histologie tumorale'),
 (178, 179, 'Hypertension essentielle (primitive) (I10)'),
 (788, 792, 'Hypertension essentielle (primitive) (I10)'),
 (1508, 1516, 'Hypertension essentielle (primitive) (I10)'),
 (589, 590, 'Hémiplégie, sans précision (G819)'),
 (834, 837, 'Hémiplégie, sans précision (G819)'),
 (839, 849, 'Hémiplégie, sans précision (G819)'),
 (1421, 1422, 'Hémiplégie, sans précision (G819)'),
 (498, 498, 'Médicaments antitumoraux'),
 (548, 548, 'Médicaments antitumoraux'),
 (727, 727, 'Médicaments antitumoraux'),
 (735, 735, 'Médicaments antitumoraux'),
 (738, 738, 'Médicaments antitumorau

### Create chunck of texts for models with limited number of tokens

In [ ]:
def get_offset(nlp, annotations, text, max_len=384):
  """
  Split the text so that the total length of labels and texts doesn't exceed `max_len`.
  """
  doc = nlp(text)
  sentences = list(doc.sents)
  split_texts = []
  stop = 0

  while stop < len(sentences):
      for i in range(len(sentences), stop, -1):
          sub_text = "".join(sent.text for sent in sentences[stop:i])
          tokenized_subtext, entity_spans, list_labels = process_text(annotations, sub_text)

          if count_tokens(list_labels, sub_text, max_len=max_len):
              split_texts.append({"tokenized_text": tokenized_subtext, "ner": entity_spans})
              stop = i
              break

  return split_texts

In [ ]:
# create chunck of texts
import spacy
i=0
text = df_gen_reports.clinical_note.iloc[i]
nlp = spacy.blank('fr')
nlp.add_pipe('sentencizer')
doc = nlp(text)
sentences = list(doc.sents)
sub_text = sentences[0].text
stop = 0

annotations = {**df_gen_reports.iloc[i]["annot_diagnostics"], **df_gen_reports.iloc[i]["annot_criteria"]}
tokenized_subtext, entity_spans, list_labels = process_text(annotations, sub_text)

In [ ]:
sub_text

'HÔPITAL DE HAUTEPIERRE - HÔPITAUX UNIVERSITAIRES DE STRASBOURG\nSERVICE DE RADIOTHÉRAPIE\n\n\nIDENTITÉ DU PATIENT\nNom : RUPPA\nPrénom : Emelia\nDate de naissance : 23/02/1963\n\n\n\nMadame Emelia Ruppa, 61 ans, est hospitalisée du 13/09/2024 au 18/09/2024 dans le service de radiothérapie pour la réalisation d’une séance de radiothérapie cérébrale.'

In [ ]:
list_labels

['Nom/Prénom du patient', 'Âge', 'Date entrée', 'Date de sortie']

In [ ]:
entity_spans

[(45, 49, 'Date de sortie'),
 (39, 43, 'Date entrée'),
 (29, 31, 'Nom/Prénom du patient'),
 (33, 34, 'Âge')]

In [ ]:
import spacy
from tqdm import tqdm

nlp = spacy.blank('fr')
nlp.add_pipe('sentencizer')

all_data = []

for i in tqdm(range(len(df_gen_reports))):
# for i in [0]:
  clinical_note = df_gen_reports.iloc[i]["clinical_note"]
  if clinical_note is not None:
    # Safely create annotations dictionary, handling potential None values
    annotations = {}
    diag_data = df_gen_reports.iloc[i]["annot_diagnostics"]
    if isinstance(diag_data, dict):
        annotations.update(diag_data)

    crit_data = df_gen_reports.iloc[i]["annot_criteria"]
    if isinstance(crit_data, dict):
        annotations.update(crit_data)

    # Only proceed if there are valid annotations
    if annotations:
        all_data.extend(get_offset(nlp, annotations, clinical_note, max_len=7000))

 34%|███▍      | 3383/9999 [06:02<11:48,  9.34it/s]


KeyboardInterrupt: 

In [ ]:
# len(retained_entities)

In [ ]:
def save_data_to_file(data, filepath):
  with open(filepath, 'w', encoding="utf-8") as f:
    json.dump(data, f)

In [ ]:
save_data_to_file(all_data, path_ref + "modern_data_1000_ner.json")